In [ ]:
import sys
print(f"{sys.version = }\n")

In [ ]:
# The classics
import numpy as np
import matplotlib.pylab as plt
import matplotlib # To get the version

In [ ]:
import zfit
from particle import Particle
from decaylanguage import DecFileParser, DecayChainViewer, DecayChain, DecayMode
import tensorflow as tf

from phasespace.fromdecay import GenMultiDecay

In [ ]:
from pprint import pprint

In [ ]:
parser = DecFileParser('/home/samyak/micromamba/envs/pyhep/lib/python3.11/site-packages/tests/fromdecay/example_decays.dec')
parser.parse()

In [ ]:
pi0_chain = parser.build_decay_chains("pi0")
pprint(pi0_chain)

In [ ]:
DecayChainViewer(pi0_chain)

In [ ]:
dplus_decay = DecayMode(1, "K- pi+ pi+ pi0", model="PHSP")
pi0_decay = DecayMode(1, "gamma gamma")
dplus_single = DecayChain("D+", {"D+": dplus_decay, "pi0": pi0_decay})
DecayChainViewer(dplus_single.to_dict())

# Generating a GenMultiDecayObject

In [ ]:
pi0_decay = GenMultiDecay.from_dict(pi0_chain)

for probability, particle in pi0_decay.gen_particles:
    print(f"There is a probability of {probability} "
          f"that pi0 decays into {', '.join(child.name for child in particle.children)}")

In [ ]:
weights, events = pi0_decay.generate(n_events=10_000)
print("Number of events for each decay mode:", ", ".join(str(len(w)) for w in weights))

# Changing Mass Settings

## Constant vs Variable Mass

In [ ]:
dsplus_chain = parser.build_decay_chains("D*+", stable_particles=["D+"])
DecayChainViewer(dsplus_chain)

In [ ]:
print(f"pi0 width = {Particle.from_evtgen_name('pi0').width}\n"
      f"D0 width = {Particle.from_evtgen_name('D0').width}")

In [ ]:
dstar_decay = GenMultiDecay.from_dict(dsplus_chain, tolerance=1e-8)
# Loop over D0 and pi+ particles, see graph above
for particle in dstar_decay.gen_particles[0][1].children:
    # If a particle width is less than tolerance or if it does not have any children, its mass will be fixed.
    assert particle.has_fixed_mass

# Loop over D+ and pi0. See above.
for particle in dstar_decay.gen_particles[1][1].children:
    if particle.name == "pi0":
        assert not particle.has_fixed_mass

# Configuring Mass Options

In [ ]:
dsplus_custom_mass_func = dsplus_chain.copy()
dsplus_chain_subset = dsplus_custom_mass_func["D*+"][1]["fs"][1]
print("Before:")
pprint(dsplus_chain_subset)
# Set the mass function of pi0 to a gaussian distribution when it decays into two photons (gamma)
dsplus_chain_subset["pi0"][0]["zfit"] = "gauss"
print("After:")
pprint(dsplus_chain_subset)

In [ ]:
dplus_decay = DecayMode(1, "K- pi+ pi+ pi0", model="PHSP")  # The model parameter will be ignored by GenMultiDecay
pi0_decay = DecayMode(1, "gamma gamma", zfit="gauss")   # Make pi0 have a gaussian mass distribution
dplus_single = DecayChain("D+", {"D+": dplus_decay, "pi0": pi0_decay})
GenMultiDecay.from_dict(dplus_single.to_dict())

In [ ]:
def custom_gauss(mass, width):
    particle_mass = tf.cast(mass, tf.float64)
    particle_width = tf.cast(width, tf.float64)

    # This is the actual mass function that will be returned
    def mass_func(min_mass, max_mass, n_events):
        min_mass = tf.cast(min_mass, tf.float64)
        max_mass = tf.cast(max_mass, tf.float64)
        # Use a zfit PDF
        pdf = zfit.pdf.Gauss(mu=particle_mass, sigma=particle_width, obs="")
        iterator = tf.stack([min_mass, max_mass], axis=-1)
        return tf.vectorized_map(
            lambda lim: pdf.sample(1, limits=(lim[0], lim[1])), iterator
        )

    return mass_func

In [ ]:
dsplus_chain_subset = dsplus_custom_mass_func["D*+"][1]["fs"][1]
print("Before:")
pprint(dsplus_chain_subset)

# Set the mass function of pi0 to the custom gaussian distribution
#  when it decays into an electron-positron pair and a photon (gamma)
dsplus_chain_subset["pi0"][1]["zfit"] = "custom_gauss"
print("After:")
pprint(dsplus_chain_subset)

In [ ]:
DecayChainViewer(dsplus_chain_subset)